In [ ]:
import logging
import gaps_online as go
#print (os.environ['DATABASE_URL'])
import cmocean
import matplotlib
import numpy as np
import tqdm
#print (go.__file__)
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import charmingbeauty as cb
import charmingbeauty.layout as lo
import dashi as d 
d.visual()
cb.visual.set_style_default()

os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = '1'
os.environ['RUST_LOG'] = 'OFF'
#go.io.dsi_j_pid_map
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)

In [ ]:
runs = {  72 : {
          'start' : 1720841217,
          'end'   : 1720880526
            },
          84 : {
          'start' : 1721372490,
          'end'   : 1721395236
            },
          86 : {
          'start' : 1721439934,
          'end'   : 1721486777
            },
         104 : {
          'start' : 1721709419,
          'end'   : 1721743848
            },
         114 : {
          'start' :  1722314183,
          'end'   :  1722352818,
         },
         120 : {
          'start' : 1722314183,
          'end'   : 1722352818
            },
         134 : {
          'start' : 1722723121,
          'end'   : 1722791305
         },
         # this is the run with the missing 85% of events
         30141 : {
          'start' : 1723693723,
          'end'   : 1723728058
            },
         30198 : {
          'start' : 1723783316,
          'end'   : 1723798041
            }
}

run_id     = 30141
#run_id     = 84
#run_id     = 134
# whole campaign
campaign_data   = {'files' : go.io.get_telemetry_binaries(runs[72]['start'], runs[30198]['end'], data_dir='/data0/gaps/csbf/csbf-data/binaries/ethernet')}
# single run
run_data   = {'files' : go.io.get_telemetry_binaries(runs[run_id]['start'], runs[run_id]['end'], data_dir='/data0/gaps/csbf/csbf-data/binaries/ethernet')}
#tof_status = go.run.get_run_tof_status(run_data['files'], check_rbs=True)

events     = go.run.load_run_from_telemetry(runs[run_id]['start'], runs[run_id]['end'])

tof_stream = go.io.get_tof_binaries(run_id, data_dir='/data0/gaps/csbf/csbf-data/')
if run_id  == 104:
    # cut off the first files, since they won't have merged event data for this run
    tof_stream = tof_stream[89:]

## Check TOF charge and peaks

In [ ]:
from_tofstream = True
reprocess      = True
nerrors        = 0
npackets       = 0
#4312015
charges_a = {k:[] for k in range(1,161)}
charges_b = {k:[] for k in range(1,161)}
peaks_a   = {k:[] for k in range(1,161)}
peaks_b   = {k:[] for k in range(1,161)}

if reprocess:
    charges_a_r = {k:[] for k in range(1,161)}
    charges_b_r = {k:[] for k in range(1,161)}
    peaks_a_r   = {k:[] for k in range(1,161)}
    peaks_b_r   = {k:[] for k in range(1,161)}
    settings = go.liftof.LiftofSettings()
    settings = settings.from_file('settings.toml')
    

if from_tofstream:
    done = False
    for j,f in tqdm.tqdm([k for k in enumerate(tof_stream)], desc='Unpacking stream...'):
        if done:
            break
        reader = go.io.TofPacketReader(str(f), filter=go.io.TofPacketType.TofEvent)
        for pack in reader:
            #print (dir(ev))
            ev = go.events.TofEvent()
            try:
                ev.from_tofpacket(pack)
            except Exception as e:
                nerrors += 1
                done = True
                print(f'Got exception {e}')
                break
                continue

            if reprocess:
                newev = go.liftof.waveform_analysis(ev, settings)
                for h in ev.hits:
                    charges_a_r[h.paddle_id].append(h.charge_a)
                    charges_b_r[h.paddle_id].append(h.charge_b)
                    peaks_a_r[h.paddle_id].append(h.peak_a)
                    peaks_b_r[h.paddle_id].append(h.peak_b)
                
            
            npackets += 1
            #if npackets == 100000:
            #    done = True
            #    print('Reachecd required number of events!')
            #    break
            #for rbev in ev.rbevents:
            #    if rbev.header.drs_lost_trigger:
            #        print ('----')
            #       print (rbev.header.drs_lost_trigger)
            #        print (rbev.header)
            #        print (ev)
            #        done = True
            #        raise
            #        break
            for h in ev.hits:
                charges_a[h.paddle_id].append(h.charge_a)
                charges_b[h.paddle_id].append(h.charge_b)
                peaks_a[h.paddle_id].append(h.peak_a)
                peaks_b[h.paddle_id].append(h.peak_b)
        #print (h)
    print (f'During reading of the TOF stream, we read {npackets} packets and encountered {nerrors} errors!')
else:
    for ev in tqdm.tqdm(events['events']):
        for h in ev.tof.hits:
            charges_a[h.paddle_id].append(h.charge_a)
            charges_b[h.paddle_id].append(h.charge_b)
            peaks_a[h.paddle_id].append(h.peak_a)
            peaks_b[h.paddle_id].append(h.peak_b)
        #print (h)
    #print (ev)
    #break


In [ ]:
%matplotlib inline
# identify large peaks
pids = range(1,161)
#bins = np.linspace(0,1000,100)
peak_bins   = np.linspace(0, 200,100)
charge_bins = np.linspace(0, 50, 100)
#for k in tqdm.tqdm([j for j in pids]):
for k in pids:
    peaks_a_pid = np.array(peaks_a[k])
    peaks_b_pid = np.array(peaks_b[k])
    print(f'-> for paddle {k} A side, we have {len(peaks_a_pid)} and {len(peaks_a_pid[peaks_a_pid > 1000])} peaks larger than 1000mV!')
    print(f'-> for paddle {k} B side, we have {len(peaks_b_pid)} and {len(peaks_b_pid[peaks_a_pid > 1000])} peaks larger than 1000mV!')
    charge_a_pid = np.array(charges_a[k])
    charge_b_pid = np.array(charges_b[k])
    
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h_a = d.factory.hist1d(peaks_a_pid, peak_bins)
    h_b = d.factory.hist1d(peaks_b_pid, peak_bins)
    h_a.line(filled=True, alpha=0.7, label='A side', color='tab:blue')
    h_b.line(filled=True, alpha=0.7, label='B side', color='tab:red')
    #h_b.line(filled=True, alpha=0.7, label='foo', color='r')
    plt.legend(loc='upper right', frameon=False)
    ax.set_ylim(bottom=0)
    #ax.set_text(f'N : {len(peaks_a_pid}')
    ax.text(0.7, 0.65, f'N(a) : {len(peaks_a_pid)}', transform=fig.transFigure)
    ax.text(0.7, 0.58, f'N(b) : {len(peaks_b_pid)}', transform=fig.transFigure)
    ax.set_title(f'Peak value [mV] Paddle {k}', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('hits', loc='top')
    ax.set_xlabel('mV', loc='right')
    fig.savefig(f'plots_{run_id}/peaks_{k}.webp')

    # charge
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h_a = d.factory.hist1d(charge_a_pid, charge_bins)
    h_b = d.factory.hist1d(charge_b_pid, charge_bins)
    h_a.line(filled=True, alpha=0.7, label='A side', color='tab:blue')
    h_b.line(filled=True, alpha=0.7, label='B side', color='tab:red')
    #h_b.line(filled=True, alpha=0.7, label='foo', color='r')
    plt.legend(loc='upper right', frameon=False)
    ax.set_ylim(bottom=0)
    #ax.set_text(f'N : {len(peaks_a_pid}')
    ax.text(0.7, 0.65, f'N(a) : {len(charge_a_pid)}', transform=fig.transFigure)
    ax.text(0.7, 0.58, f'N(b) : {len(charge_b_pid)}', transform=fig.transFigure)
    ax.set_title(f'Charge [pC] Paddle {k}', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('hits', loc='top')
    ax.set_xlabel('pC', loc='right')
    fig.savefig(f'plots_{run_id}/charge_{k}.webp')

    # peak vs charge
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h   = d.factory.hist2d((peaks_a_pid,charge_a_pid), (peak_bins, charge_bins))
    h.imshow(cmap=matplotlib.colormaps['coolwarm'])
    ax.set_ylim(bottom=0)
    ax.set_xlim(left=0)
    ax.set_title(f'Peak vs Charge Paddle {k}, A', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('pC', loc='top')
    ax.set_xlabel('mV', loc='right')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    fig.savefig(f'plots_{run_id}/peak_vs_charge_{k}A.webp')
    

    # peak vs charge
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h   = d.factory.hist2d((peaks_b_pid,charge_b_pid), (peak_bins, charge_bins))
    h.imshow(cmap=matplotlib.colormaps['coolwarm'])
    ax.set_ylim(bottom=0)
    ax.set_xlim(left=0)
    ax.set_title(f'Peak vs Charge Paddle {k}, B', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('Charge in pC', loc='top')
    ax.set_xlabel('Peak in mV', loc='right')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    fig.savefig(f'plots_{run_id}/peak_vs_charge_{k}B.webp')

    # A vs B
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h   = d.factory.hist2d((charge_a_pid,charge_b_pid), (charge_bins, charge_bins))
    h.imshow(cmap=matplotlib.colormaps['coolwarm'])
    ax.set_ylim(bottom=0)
    ax.set_xlim(left=0)
    ax.set_title(f'Charge A vs B Paddle {k}', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('Charge (B) in pC', loc='top')
    ax.set_xlabel('Charge (A)in pC', loc='right')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    fig.savefig(f'plots_{run_id}/charge_vs_charge_{k}.webp')

    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h   = d.factory.hist2d((peaks_a_pid,peaks_b_pid), (peak_bins, peak_bins))
    h.imshow(cmap=matplotlib.colormaps['coolwarm'])
    ax.set_ylim(bottom=0)
    ax.set_xlim(left=0)
    ax.set_title(f'Peak A vs B Paddle {k}', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('Peak (B) in mV', loc='top')
    ax.set_xlabel('Peak (A) in mV', loc='right')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    fig.savefig(f'plots_{run_id}/peak_vs_peak_{k}.webp')

    
    
    #break
plt.show()
    
    #all_peaks = len(np.array(peaks_a[k])

In [ ]:
%matplotlib inline
# identify large peaks
pids = range(1,161)
#bins = np.linspace(0,1000,100)
peak_bins   = np.linspace(0, 200,100)
charge_bins = np.linspace(0, 50, 100)
#for k in tqdm.tqdm([j for j in pids]):
for k in pids:
    peaks_a_pid = np.array(peaks_a[k])
    peaks_b_pid = np.array(peaks_b[k])
    print(f'-> for paddle {k} A side, we have {len(peaks_a_pid)} and {len(peaks_a_pid[peaks_a_pid > 1000])} peaks larger than 1000mV!')
    print(f'-> for paddle {k} B side, we have {len(peaks_b_pid)} and {len(peaks_b_pid[peaks_a_pid > 1000])} peaks larger than 1000mV!')
    charge_a_pid = np.array(charges_a[k])
    charge_b_pid = np.array(charges_b[k])
    charge_a_r_pid = np.array(charges_a_r[k])
    charge_b_r_pid = np.array(charges_b_r[k])

    
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h_a = d.factory.hist1d(peaks_a_pid, peak_bins)
    h_b = d.factory.hist1d(peaks_b_pid, peak_bins)
    h_a.line(filled=True, alpha=0.7, label='A side', color='tab:blue')
    h_b.line(filled=True, alpha=0.7, label='B side', color='tab:red')
    #h_b.line(filled=True, alpha=0.7, label='foo', color='r')
    plt.legend(loc='upper right', frameon=False)
    ax.set_ylim(bottom=0)
    #ax.set_text(f'N : {len(peaks_a_pid}')
    ax.text(0.7, 0.65, f'N(a) : {len(peaks_a_pid)}', transform=fig.transFigure)
    ax.text(0.7, 0.58, f'N(b) : {len(peaks_b_pid)}', transform=fig.transFigure)
    ax.set_title(f'Peak value [mV] Paddle {k}', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('hits', loc='top')
    ax.set_xlabel('mV', loc='right')
    fig.savefig(f'plots_{run_id}/peaks_{k}_r.webp')

    # charge
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h_a = d.factory.hist1d(charge_a_r_pid, charge_bins)
    h_b = d.factory.hist1d(charge_b_r_pid, charge_bins)
    h_a.line(filled=True, alpha=0.7, label='A side', color='tab:blue')
    h_b.line(filled=True, alpha=0.7, label='B side', color='tab:red')
    #h_b.line(filled=True, alpha=0.7, label='foo', color='r')
    plt.legend(loc='upper right', frameon=False)
    ax.set_ylim(bottom=0)
    #ax.set_text(f'N : {len(peaks_a_pid}')
    ax.text(0.7, 0.65, f'N(a) : {len(charge_a_r_pid)}', transform=fig.transFigure)
    ax.text(0.7, 0.58, f'N(b) : {len(charge_b_r_pid)}', transform=fig.transFigure)
    ax.set_title(f'Charge [pC] Paddle {k}', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('hits', loc='top')
    ax.set_xlabel('pC', loc='right')
    fig.savefig(f'plots_{run_id}/charge_{k}_r.webp')

    # peak vs charge
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h   = d.factory.hist2d((peaks_a_pid,charge_a_r_pid), (peak_bins, charge_bins))
    h.imshow(cmap=matplotlib.colormaps['coolwarm'])
    ax.set_ylim(bottom=0)
    ax.set_xlim(left=0)
    ax.set_title(f'Peak vs Charge Paddle {k}, A', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('pC', loc='top')
    ax.set_xlabel('mV', loc='right')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    fig.savefig(f'plots_{run_id}/peak_vs_charge_{k}A_r.webp')
    

    # peak vs charge
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h   = d.factory.hist2d((peaks_b_pid,charge_b_r_pid), (peak_bins, charge_bins))
    h.imshow(cmap=matplotlib.colormaps['coolwarm'])
    ax.set_ylim(bottom=0)
    ax.set_xlim(left=0)
    ax.set_title(f'Peak vs Charge Paddle {k}, B', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('Charge in pC', loc='top')
    ax.set_xlabel('Peak in mV', loc='right')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    fig.savefig(f'plots_{run_id}/peak_vs_charge_{k}B_r.webp')

    # A vs B
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h   = d.factory.hist2d((charge_a_r_pid,charge_b_r_pid), (charge_bins, charge_bins))
    h.imshow(cmap=matplotlib.colormaps['coolwarm'])
    ax.set_ylim(bottom=0)
    ax.set_xlim(left=0)
    ax.set_title(f'Charge A vs B Paddle {k}', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('Charge (B) in pC', loc='top')
    ax.set_xlabel('Charge (A)in pC', loc='right')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    fig.savefig(f'plots_{run_id}/charge_vs_charge_{k}_r.webp')

    # reprocessed vs "old"
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h   = d.factory.hist2d((charge_a_pid,charge_a_r_pid), (charge_bins, charge_bins))
    h.imshow(cmap=matplotlib.colormaps['coolwarm'])
    ax.set_ylim(bottom=0)
    ax.set_xlim(left=0)
    ax.set_title(f'Charge repr. vs nominal {k}A', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('Charge (repr) in pC', loc='top')
    ax.set_xlabel('Charge in pC', loc='right')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    fig.savefig(f'plots_{run_id}/charge_repr_vs_nominal_{k}A.webp')

    # reprocessed vs "old"
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h   = d.factory.hist2d((charge_b_pid,charge_b_r_pid), (charge_bins, charge_bins))
    h.imshow(cmap=matplotlib.colormaps['coolwarm'])
    ax.set_ylim(bottom=0)
    ax.set_xlim(left=0)
    ax.set_title(f'Charge repr. vs nominal {k}B', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('Charge (repr) in pC', loc='top')
    ax.set_xlabel('Charge in pC', loc='right')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    fig.savefig(f'plots_{run_id}/charge_repr_vs_nominal_{k}B.webp')

    
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    h   = d.factory.hist2d((peaks_a_pid,peaks_b_pid), (peak_bins, peak_bins))
    h.imshow(cmap=matplotlib.colormaps['coolwarm'])
    ax.set_ylim(bottom=0)
    ax.set_xlim(left=0)
    ax.set_title(f'Peak A vs B Paddle {k}', loc='right')
    #ax.set_yscale('symlog')
    ax.set_ylabel('Peak (B) in mV', loc='top')
    ax.set_xlabel('Peak (A) in mV', loc='right')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    fig.savefig(f'plots_{run_id}/peak_vs_peak_{k}_r.webp')

    
    
    #break
plt.show()
    
    #all_peaks = len(np.array(peaks_a[k])

----------------

In [ ]:
npacks = 0
done   = False
for f in events['files']:
    if done:
        break
    reader = go.io.TelemetryPacketReader(str(events['files'][0]), filter=go.io.TelemetryPacketType.MergedEvent)
    
    for pack in reader:
        me = go.events.MergedEvent()
        try:
            me.from_telemetrypacket(pack)
        except Exception as e:
            print (pack)
            print (f'error! {e}')
            print (pack.payload)
            done = True
            break
        try:
            me.tof
            #done = True
        except Exception as e:
            #continue 
            print (pack)
            print (f'TOF error! {e}')
            print (pack.payload)
            print (me)
            print (me.tracker_v2)
            done = True
            break    
        npacks += 1

In [ ]:
print (npacks)
print (me)

In [ ]:
tofeventbytestream = [ 170, 170, 23, 102, 0, 0, 0, 170, 170, 0, 0, 4, 2, 104, 141, 118, 0, 0, 16, 205, 234, 145, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 2, 0, 64, 0, 128, 32, 0, 0, 0, 16, 0, 0, 0, 2, 0, 240, 240, 105, 59, 87, 55, 87, 59, 77, 25, 80, 157, 70, 191, 73, 0, 0, 57, 57, 137, 58, 52, 71, 64, 47, 57, 83, 58, 15, 15, 240, 240, 55, 136, 87, 126, 87, 129, 93, 43, 93, 19, 83, 118, 82, 0, 0, 20, 188, 144, 60, 145, 72, 64, 236, 183, 218, 59, 15, 15, 85, 85, 85, 85]

In [ ]:
37501783/500/3600

In [ ]:
t_reader = go.io.TofPacketReader(str(tof_stream[0]), filter=go.io.TofPacketType.TofEvent)
for pack in t_reader:
    ev = go.events.TofEvent()
    ev.from_tofpacket(pack)
    print (ev.mastertriggerevent)
    break

In [ ]:
import go_pybindings as gop
print (len(tofeventbytestream))
tofevent = go.io.TofPacket()
tofevent.from_bytestream(tofeventbytestream, 0)
tofsummary = gop.events.TofEventSummary()
tofsummary.from_tofpacket(tofevent)
print (tofsummary)

In [ ]:
evids_telly = []
for ev in tqdm.tqdm(events['events']):
    #print (ev)
    ev.tof.event_id
    evids_telly.append(ev.tof.event_id)
    #break

In [ ]:
evids_toffy = []
npackets    = 0
done        = False
for j,f in tqdm.tqdm([k for k in enumerate(tof_stream)], desc='Unpacking stream...'):
    if done:
        break
    reader = go.io.TofPacketReader(str(f), filter=go.io.TofPacketType.TofEvent)
    for pack in tqdm.tqdm(reader):
        #print (dir(ev))
        ev = go.events.TofEvent()
        ev.from_tofpacket(pack)
        npackets += 1
        evids_toffy.append(ev.event_id)
        #if npackets == 100000:
        #    done = True
        #    break
        #if ev.event_id < 7769451:
        #    continue


In [ ]:
print (len(evids_toffy))
print (len(evids_telly))
print(evids_telly[:10])
print(evids_toffy[:100])

def check_missing_events(evids : list) -> None:
    """
    Compare the list of event ids to a list with 
    consecutive rising event ids for the same range

    # Arguments:
        evids  : A list of event ids
    """
    all_evids = range(evids[0], evids[-1])
    print (f'-> We found {len(evids)} event ids when expecting {len(all_evids)}')
    print (f'-> We are missing {100*(len(all_evids) - len(evids))/len(all_evids):.3f}%')

check_missing_events(evids_toffy)
check_missing_events(evids_telly)


In [ ]:
paddle = go.db.Paddle.objects.filter(paddle_id=19)
paddle

In [ ]:
tof_stream_peaks_a = []
tof_stream_peaks_b = []
tof_stream_times_a = []
tof_stream_times_b = []
hits               = []
stati              = []
wfs = { k : [] for k in range(1,9)}
for j,f in enumerate(tof_stream):
    reader = go.io.TofPacketReader(str(f), filter=go.io.TofPacketType.TofEvent)
    for pack in tqdm.tqdm(reader):
        #print (dir(ev))
        ev = go.events.TofEvent()
        ev.from_tofpacket(pack)
        check_wf = False
        for h in ev.hits:
            if h.paddle_id == 19:
                if h.peak_a > 3500:
                    check_wf = True
                    hits.append(h)
                    stati.append(ev.get_summary().status)
                    tof_stream_peaks_a.append(h.peak_a)
                    tof_stream_peaks_b.append(h.peak_b)
                    tof_stream_times_a.append(h.time_a)
                    tof_stream_times_b.append(h.time_b)
                    break
                    
        if check_wf:
            for wf in ev.waveforms:
                #print(wf.rb_id, wf.rb_channel)
                if wf.rb_channel == 8:
                    continue
                if go.io.rb_pid_map[wf.rb_id][wf.rb_channel + 1] == 19:
                    wfs[wf.rb_channel + 1].append(wf)
                    #wfs_a.append(wf)
                    #if wf.rb_channel == 4:
                    #    wfs_b.append(wf)
                        
    if j == 10:
        #break
        pass
                    
 
                

In [ ]:
ev.get_summary().status

In [ ]:
print(len(tof_stream_peaks_a))
print(len(tof_stream_peaks_b))
print (wfs.keys())
for k in wfs.keys():
    print (k, len(wfs[k]))


In [ ]:
import go_pybindings as gop
cali = gop.events.RBCalibration()
cali.from_file('/data0/gaps/csbf/csbf-data/calib/240702_215856UTC/RB41_240702_215905UTC.cali.tof.gaps')


In [ ]:
idx = 1
for idx in range(4):
    print(tof_stream_peaks_a[idx])
    print(tof_stream_peaks_b[idx])
    print(tof_stream_times_a[idx])
    print(tof_stream_times_b[idx])
    print (hits[idx])
    print (stati[idx])
    wfs[3][idx].calibrate(cali)
    wfs[4][idx].calibrate(cali)
    #wfs_b[idx].calibrate(cali)
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    ax.text(300, 100,f'{hits[idx]}', fontsize=7)#, transform=matplotlib.transform.TransFigure)
    ax.plot(wfs[3][idx].times,wfs[3][idx].voltages)
    ax.plot(wfs[4][idx].times,wfs[4][idx].voltages)
    ax.set_ylim(bottom=-5)
    #ax.plot(wfs_b[idx].times,wfs_b[idx].voltages)
    plt.show()

In [ ]:

def find_paddle(hit, paddles):
    # hit is dsi, ch, channel
    for pdl in paddles:
        if pdl.dsi == hit[0]:
            if pdl.j_ltb == hit[1]:
                if pdl.ltb_chA == hit[2][0]:
                    return pdl.paddle_id
                elif pdl.ltb_chB == hit[2][0]:
                    return pdl.paddle_id
    print (f'No paddle found for {hit}')
    
def create_occupancy_dict(events, normalize=True, use_trigger_hits=False):
    if use_trigger_hits:
        paddles = go.db.get_tof_paddles()
    occu_per_paddle = {k : 0 for k in range(1,161)}
    for ev in tqdm.tqdm(events, desc='Getting TOF occupancy data!'):
        if use_trigger_hits:
            for h in ev.tof.trigger_hits:
                pid = find_paddle(h, paddles)
                occu_per_paddle[pid] += 1

        else:
            for h in ev.tof.hits:
                occu_per_paddle[h.paddle_id] += 1
    # normalize it
    if normalize:
        max_val = max(occu_per_paddle.values())
        for k in occu_per_paddle.keys():
            occu_per_paddle[k] = occu_per_paddle[k]/max_val
    return occu_per_paddle

In [ ]:
import numpy as np
def peak_height(events, paddle_id=19):
    """
    Plot the peak height for the TofEventSummary
    packets
    """
    peaks = []
    for ev in tqdm.tqdm(events):
        for h in ev.tof.hits:
            if h.paddle_id == paddle_id:
                #print (dir(h))
                peaks.append(h.peak_a)
    return np.array(peaks)
    

In [ ]:
peaks = peak_height(events['events'])

In [ ]:
print (events.keys())

In [ ]:
import charmingbeauty.layout as lo
import dashi as d
d.visual()
print (len(peaks))

fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
ax  = fig.gca()
h   = d.factory.hist1d(peaks, 100)
h.line(color='b',filled=True, alpha=0.5)
ax.set_yscale('log')
ax.set_xlabel('peak height [mV]', loc='right')
ax.set_ylabel('nhits', loc='top')
ax.set_title('Paddle 1')
plt.show()

In [ ]:
def occupancy_xy(occu_per_paddle : dict, 
                 cmap = matplotlib.colormaps['hot']):
    """
    Make an occupancy plot for all panels which 
    are facing in z-direction
    These are the whole Umbrella as well as 
    CBE TOP + Bottom
    
    # Arguments:
        occu_per_paddle : paddle_id -> normalized occupancy

    # Keyword Arguments:
        cmap            : colormap
    """
    fig, axs = plt.subplots(1, 3, figsize=(18, 5), gridspec_kw={'width_ratios': [1, 1, 1]})
    umb_paddles = go.db.get_umbrella_paddles()
    cbe_top_paddles = go.db.Paddle.objects.filter(panel_id=1)
    cbe_bot_paddles = go.db.Paddle.objects.filter(panel_id=2)
    xmin, xmax = -100,100
    ymin, ymax = -100,100
    zmin, zmax = -25, 120
    title      = 'Relative occupancy, xy projection'
    for pdl in umb_paddles:
        #continue
        #don't plot cb bottom
        #if pdl.paddle_id in range(1,12):
        #if pdl.paddle_id in range(13,24):
        #    continue
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[0].add_patch(pdl.draw_xy(fill=True, edgecolor=color, facecolor=color))
        axs[0].set_xlabel('x [cm]', loc='right')
        axs[0].set_ylabel('y [cm]', loc='top')#, rotation=90)
        axs[0].set_aspect('equal')
        axs[0].set_xlim(-200, 200)
        axs[0].set_ylim(-200, 200)
        axs[0].set_title('UMB', loc='right')
    for pdl in cbe_top_paddles:   
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[1].add_patch(pdl.draw_xy(fill=True, edgecolor=color, facecolor=color))
        axs[1].set_xlabel('x [cm]', loc='right')
        axs[1].set_ylabel('y [cm]', loc='top')#, rotation=90)
        axs[1].set_xlim(-100, 100)
        axs[1].set_ylim(-100, 100)
        axs[1].set_aspect('equal')
        axs[1].set_title('CBE TOP', loc='right')
    for pdl in cbe_bot_paddles:
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[2].add_patch(pdl.draw_xy(fill=True, edgecolor=color, facecolor=color))
        axs[2].set_xlabel('x [cm]', loc='right')
        axs[2].set_ylabel('y [cm]', loc='top')#, rotation=270)
        axs[2].set_xlim(-100, 100)
        axs[2].set_ylim(-100, 100)
        axs[2].set_aspect('equal')
        axs[2].set_title('CBE BOT')
        
    axs[0].spines['top'].set_visible(True)
    axs[1].spines['top'].set_visible(True)
    axs[2].spines['top'].set_visible(True)
    axs[0].spines['right'].set_visible(True)
    axs[1].spines['right'].set_visible(True)
    axs[2].spines['right'].set_visible(True)
    cbar_ax = fig.add_axes([0.9, 0.0, 0.05, 1.0])
    cbar_ax.set_axis_off()
    sm = cm.ScalarMappable(cmap=cmap, norm=matplotlib.colors.Normalize())
    sm.set_array([0, 1])
    ax = plt.sca(cbar_ax)
    plt.colorbar(sm, ax=cbar_ax, label='Relative occupancy')
    fig.suptitle(title, x=0.9)
    return fig, axs

def occupancy_yz(occu_per_paddle : dict, 
                 cmap = matplotlib.colormaps['hot']):
    """
    Make an occupancy plot for all panels which 
    are facing in z-direction
    These are the whole Umbrella as well as 
    CBE TOP + Bottom
    
    # Arguments:
        occu_per_paddle : paddle_id -> normalized occupancy

    # Keyword Arguments:
        cmap            : colormap
    """
    fig, axs = plt.subplots(1, 4, figsize=(22, 5), gridspec_kw={'width_ratios': [1, 1, 1, 1]})
    #umb_paddles = go.db.get_umbrella_paddles()
    #cbe_top_paddles = go.db.Paddle.objects.filter(panel_id=1)
    #cbe_bot_paddles = go.db.Paddle.objects.filter(panel_id=2)
    cbe_front = go.db.Paddle.objects.filter(panel_id=3) # cube front
    cbe_back  = go.db.Paddle.objects.filter(panel_id=5) # cube back
    cor_front = go.db.Paddle.objects.filter(panel_id=14) # cor front
    cor_back  = go.db.Paddle.objects.filter(panel_id=16) # cor back
    xmin, xmax = -100,100
    ymin, ymax = -100,100
    zmin, zmax = -25, 120
    title      = 'Relative occupancy, xy projection'
    for pdl in cbe_front:
        #continue
        #don't plot cb bottom
        #if pdl.paddle_id in range(1,12):
        #if pdl.paddle_id in range(13,24):
        #    continue
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[0].add_patch(pdl.draw_yz(fill=True, edgecolor=color, facecolor=color))
        axs[0].set_xlabel('x [cm]', loc='right')
        axs[0].set_ylabel('y [cm]', loc='top')#, rotation=90)
        axs[0].set_aspect('equal')
        axs[0].set_xlim(-80, 80)
        axs[0].set_ylim(-10, 120)
        axs[0].set_title('CBE +X', loc='right')
    for pdl in cbe_back:   
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[1].add_patch(pdl.draw_yz(fill=True, edgecolor=color, facecolor=color))
        axs[1].set_xlabel('x [cm]', loc='right')
        axs[1].set_ylabel('y [cm]', loc='top')#, rotation=90)
        axs[1].set_xlim(-80, 80)
        axs[1].set_ylim(-10, 120)
        axs[1].set_aspect('equal')
        axs[1].set_title('CBE -X', loc='right')
    for pdl in cor_front:   
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[2].add_patch(pdl.draw_yz(fill=True, edgecolor=color, facecolor=color))
        axs[2].set_xlabel('x [cm]', loc='right')
        axs[2].set_ylabel('y [cm]', loc='top')#, rotation=90)
        axs[2].set_xlim(-100, 100)
        axs[2].set_ylim(-50, 180)
        axs[2].set_aspect('equal')
        axs[2].set_title('COR +X', loc='right')
    for pdl in cor_back:   
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[3].add_patch(pdl.draw_yz(fill=True, edgecolor=color, facecolor=color))
        axs[3].set_xlabel('x [cm]', loc='right')
        axs[3].set_ylabel('y [cm]', loc='top')#, rotation=90)
        axs[3].set_xlim(-100,100)
        axs[3].set_ylim(-50, 180)
        axs[3].set_aspect('equal')
        axs[3].set_title('COR -X', loc='right')
    #for pdl in cbe_bot_paddles:
    #    color = cmap(occu_per_paddle[pdl.paddle_id])
    #    axs[2].add_patch(pdl.draw_xy(fill=True, edgecolor=color, facecolor=color))
    #    axs[2].set_xlabel('x [cm]', loc='right')
    #    axs[2].set_ylabel('y [cm]', loc='top')#, rotation=270)
    #    axs[2].set_xlim(-100, 100)
    #    axs[2].set_ylim(-100, 100)
    #    axs[2].set_aspect('equal')
    #    axs[2].set_title('CBE BOT')
        
    axs[0].spines['top'].set_visible(True)
    axs[1].spines['top'].set_visible(True)
    axs[2].spines['top'].set_visible(True)
    axs[3].spines['top'].set_visible(True)
    axs[0].spines['right'].set_visible(True)
    axs[1].spines['right'].set_visible(True)
    axs[2].spines['right'].set_visible(True)
    axs[3].spines['right'].set_visible(True)
    cbar_ax = fig.add_axes([0.9, 0.0, 0.05, 1.0])
    cbar_ax.set_axis_off()
    sm = cm.ScalarMappable(cmap=cmap, norm=matplotlib.colors.Normalize())
    sm.set_array([0, 1])
    ax = plt.sca(cbar_ax)
    plt.colorbar(sm, ax=cbar_ax, label='Relative occupancy')
    fig.suptitle(title, x=0.9)
    return fig, axs

def occupancy_xz(occu_per_paddle : dict, 
                 cmap = matplotlib.colormaps['hot']):
    """
    Make an occupancy plot for all panels which 
    are facing in z-direction
    These are the whole Umbrella as well as 
    CBE TOP + Bottom
    
    # Arguments:
        occu_per_paddle : paddle_id -> normalized occupancy

    # Keyword Arguments:
        cmap            : colormap
    """
    fig, axs = plt.subplots(1, 4, figsize=(22, 5), gridspec_kw={'width_ratios': [1, 1, 1, 1]})
    #umb_paddles = go.db.get_umbrella_paddles()
    #cbe_top_paddles = go.db.Paddle.objects.filter(panel_id=1)
    #cbe_bot_paddles = go.db.Paddle.objects.filter(panel_id=2)
    cbe_front = go.db.Paddle.objects.filter(panel_id=4) # cube front
    cbe_back  = go.db.Paddle.objects.filter(panel_id=6) # cube back
    cor_front = go.db.Paddle.objects.filter(panel_id=15) # cor front
    cor_back  = go.db.Paddle.objects.filter(panel_id=17) # cor back
    xmin, xmax = -100,100
    ymin, ymax = -100,100
    zmin, zmax = -25, 120
    title      = 'Relative occupancy, xy projection'
    for pdl in cbe_front:
        #continue
        #don't plot cb bottom
        #if pdl.paddle_id in range(1,12):
        #if pdl.paddle_id in range(13,24):
        #    continue
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[0].add_patch(pdl.draw_xz(fill=True, edgecolor=color, facecolor=color))
        axs[0].set_xlabel('x [cm]', loc='right')
        axs[0].set_ylabel('y [cm]', loc='top')#, rotation=90)
        axs[0].set_aspect('equal')
        axs[0].set_xlim(-80, 80)
        axs[0].set_ylim(-10, 120)
        axs[0].set_title('CBE +X', loc='right')
    for pdl in cbe_back:   
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[1].add_patch(pdl.draw_xz(fill=True, edgecolor=color, facecolor=color))
        axs[1].set_xlabel('x [cm]', loc='right')
        axs[1].set_ylabel('y [cm]', loc='top')#, rotation=90)
        axs[1].set_xlim(-80, 80)
        axs[1].set_ylim(-10, 120)
        axs[1].set_aspect('equal')
        axs[1].set_title('CBE -X', loc='right')
    for pdl in cor_front:   
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[2].add_patch(pdl.draw_xz(fill=True, edgecolor=color, facecolor=color))
        axs[2].set_xlabel('x [cm]', loc='right')
        axs[2].set_ylabel('y [cm]', loc='top')#, rotation=90)
        axs[2].set_xlim(-100, 100)
        axs[2].set_ylim(-50, 180)
        axs[2].set_aspect('equal')
        axs[2].set_title('COR +X', loc='right')
    for pdl in cor_back:   
        color = cmap(occu_per_paddle[pdl.paddle_id])
        axs[3].add_patch(pdl.draw_xz(fill=True, edgecolor=color, facecolor=color))
        axs[3].set_xlabel('x [cm]', loc='right')
        axs[3].set_ylabel('y [cm]', loc='top')#, rotation=90)
        axs[3].set_xlim(-100,100)
        axs[3].set_ylim(-50, 180)
        axs[3].set_aspect('equal')
        axs[3].set_title('COR -X', loc='right')
    #for pdl in cbe_bot_paddles:
    #    color = cmap(occu_per_paddle[pdl.paddle_id])
    #    axs[2].add_patch(pdl.draw_xy(fill=True, edgecolor=color, facecolor=color))
    #    axs[2].set_xlabel('x [cm]', loc='right')
    #    axs[2].set_ylabel('y [cm]', loc='top')#, rotation=270)
    #    axs[2].set_xlim(-100, 100)
    #    axs[2].set_ylim(-100, 100)
    #    axs[2].set_aspect('equal')
    #    axs[2].set_title('CBE BOT')
        
    axs[0].spines['top'].set_visible(True)
    axs[1].spines['top'].set_visible(True)
    axs[2].spines['top'].set_visible(True)
    axs[3].spines['top'].set_visible(True)
    axs[0].spines['right'].set_visible(True)
    axs[1].spines['right'].set_visible(True)
    axs[2].spines['right'].set_visible(True)
    axs[3].spines['right'].set_visible(True)
    cbar_ax = fig.add_axes([0.9, 0.0, 0.05, 1.0])
    cbar_ax.set_axis_off()
    sm = cm.ScalarMappable(cmap=cmap, norm=matplotlib.colors.Normalize())
    sm.set_array([0, 1])
    ax = plt.sca(cbar_ax)
    plt.colorbar(sm, ax=cbar_ax, label='Relative occupancy')
    fig.suptitle(title, x=0.9)
    return fig, axs



In [ ]:
%matplotlib inline
plt.close('all')
plt.clf()
import charmingbeauty as cb
cb.visual.set_style_default()

#cmap = matplotlib.colormaps['hot']
#cbe_top = go.db.Paddle.objects.filter(paddle_id__lt=13)

#occu_dict = create_occupancy_dict(events['events'], use_trigger_hits=True)

#fig, axs = cube_occupancy(occu_dict, which='umbrella')
#fig.savefig('occ1.webp')
#fig, axs = cube_occupancy(occu_dict, which='cbe', paddles=cbe_top)
#fig.savefig('occ2.webp')
#fig, axs = cube_occupancy(occu_dict, which='all')
#fig.savefig('occ3.webp')
cmap = cmocean.cm.balance
cmap = matplotlib.colormaps['seismic']
fig, axs = occupancy_xy(occu_dict, cmap = cmap)
fig, axs = occupancy_yz(occu_dict, cmap = cmap)
fig, axs = occupancy_xz(occu_dict, cmap = cmap)

plt.show()


In [ ]:
print (tof_status.keys())
set(tof_status['missing_paddels'])

In [ ]:
paddles = go.db.get_cube_paddles()
print (len (paddles))

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import charmingbeauty as cb
cb.visual.set_style_default()
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
#plt.clf()
#plt.close('all')
cmap = cm.get_cmap('hot')
paddles = go.db.get_tof_paddles()
paddles = go.db.get_umbrella_paddles()
paddles = go.db.get_cube_paddles()
for pdl in paddles:
    #if pdl.panel_id < 7 or pdl.panel_id > 13:
    #    pdl.points = None
    ##if pdl.panel_id != 6:
    # #   continue
    #patches  = pdl.get_projections()
    #print (pdl.normal, 'normal')
    #patches[0].set(edgecolor='b', fill=True, lw=0.9)
    color = cmap(occu_per_paddle[pdl.paddle_id])
    axs[0].add_patch(pdl.draw_xy(fill=True, edgecolor=color, facecolor=color))
    axs[1].add_patch(pdl.draw_xz(fill=True, edgecolor=color, facecolor=color))
    axs[2].add_patch(pdl.draw_yz(fill=True, edgecolor=color, facecolor=color))
    axs[0].set_xlabel('x', loc='right')
    axs[0].set_ylabel('y', loc='top', rotation=90)
    axs[0].set_aspect('equal')
    axs[0].set_xlim(-200,200)
    axs[0].set_ylim(-200,200)
    axs[1].set_xlabel('x', loc='right')
    axs[1].set_ylabel('z', loc='top', rotation=90)
    axs[1].set_xlim(-200,200)
    axs[1].set_ylim(-50,250)
    axs[1].set_aspect('equal')
    axs[2].set_xlabel('y', loc='right')
    axs[2].set_ylabel('z', loc='top', rotation=270)
    axs[2].set_xlim(-200,200)
    axs[2].set_ylim(-50,250)
    axs[2].set_aspect('equal')
    axs[0].spines['top'].set_visible(True)
    axs[1].spines['top'].set_visible(True)
    axs[2].spines['top'].set_visible(True)
    axs[0].spines['right'].set_visible(True)
    axs[1].spines['right'].set_visible(True)
    axs[2].spines['right'].set_visible(True)

#ax = pdl.plot_projections(data)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

import numpy as np
import charmingbeauty.layout as lo
import charmingbeauty as cb
cb.visual.set_style_default()


fig  = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
ax   = fig.gca()
times = [k[0] for k in tof_status['npaddles'] if k[1] != 0]
npdl  = [k[1] for k in tof_status['npaddles'] if k[1] != 0]
times = np.array(times)
times = times/(3600*24)
ax.set_title('TOF stability during campaign', loc='right')
ax.scatter(times, npdl, s=0.8)
ax.hlines(160,times[0], times[-1],lw=1.0, color='gray')
ax.set_xlabel('MET (gcu) [days]', loc='right')
ax.set_ylabel('Active paddles')
cb.visual.adjust_minor_ticks(ax, which='both')

fig  = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
ax   = fig.gca()
ax.set_title('TOF stability during campaign', loc='right')
times = [k[0] for k in tof_status['nrbs'] if k[1] != 0]
nrbs  = [k[1] for k in tof_status['nrbs'] if k[1] != 0]
times = np.array(times)
times = times/(3600*24)
ax.scatter(times, nrbs, s=0.8)
ax.hlines(40,times[0], times[-1],lw=1.0, color='gray')
ax.set_xlabel('MET (gcu) [days]', loc='right')
ax.set_ylabel('Active RBs')
cb.visual.adjust_minor_ticks(ax, which='both')
plt.show()

In [ ]:
import gaps_online as go
tl = go.telemetry

reader = tl.TelemetryPacketReader(str(f), filter=tl.TelemetryPacketType.AnyTrackerHK)
#reader.get_packet_index(verbose=True)
reader.rewind()
for k in reader:
    temps_packet = tl.TrackerDAQTempPacket()
    hsk_packet   = tl.TrackerDAQHSKPacket()
    try:
        #print (k)
        temps_packet.from_telemetrypacket(k)
        #hsk_packet.from_telemetrypacket(k)
    except Exception as e:
        #print(e)
        print (k)
        continue
    print (temps_packet)
    #print (hsk_packet)

In [ ]:
import gaps_online as go
tl = go.telemetry

reader = tl.TelemetryPacketReader(str(f), filter=tl.TelemetryPacketType.Tracker)
reader.get_packet_index(verbose=True)
reader.rewind()
for k in reader:
    temps_packet = tl.TrackerPacket()
    try:
        temps_packet.from_telemetrypacket(k)
    except Exception as e:
        print(e)
        print (k)
        continue
    #print (temps_packet)
    print ('filtering...')
    print (temps_packet.events[0].filter_hits(lambda t : t.asic_event_code == 2 or t.asic_event_code == 0))
    break

In [ ]:
%matplotlib inline

import charmingbeauty.layout as lo
import numpy as np

cb.visual.set_style_default()
fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
ax  = fig.gca()
ax.set_ylabel('Hz', loc='top')
ax.set_xlabel('MET [s] (gcu)')#, loc='right')

times   = np.array([j[0] for j in mtbmoni])
#times  -= times[0]
rates   = np.array([j[1].rate for j in mtbmoni])
l_rates = np.array([j[1].lost_rate for j in mtbmoni])
print (times[l_rates<500][-1])
print(f'-> Avg MTB rate {rates.mean()}')
print(f'-> Avg Lost rate {l_rates.mean()}')
ax.plot(times, rates, lw=0.8, alpha=0.7, label='rate')
ax.plot(times, l_rates, lw=0.8, alpha=0.7, label='lost rate')
ax.legend(loc='upper right', frameon=False)
ax.set_title(f'MTB rates', loc='right')
cb.visual.adjust_minor_ticks(ax, which='both')
plt.show()

In [ ]:
%matplotlib inline

import dashi as d
d.visual()
import charmingbeauty.layout as lo

nhits  = [len(ev.tof.hits) for ev in tqdm.tqdm(events)]
nthits = [len(ev.tof.trigger_hits) for ev in tqdm.tqdm(events)]
nhits_int = 0
nthits_int = 0
for k in nhits:
    nhits_int += k
for k in nthits:
    nthits_int += k
print(f'Integrated missing HG hits {100*nhits_int/(nhits_int+nthits_int):4.2f}%')

In [ ]:
import charmingbeauty.layout as lo
import dashi as d
import numpy as np
d.visual()
cb.visual.set_style_default()
fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
ax  = plt.gca()
h   = d.factory.hist1d(nhits, np.arange(-0.5,30.5,1))
h2  = d.factory.hist1d(nthits, np.arange(-0.5,30.5,1))
h.line(filled=True, alpha=0.7, color='r', label='HG')
h2.line(color='b', label='LG')
ax.legend(frameon=False)
ax.set_yscale('log')
ax.set_xlabel('TOF hits', loc='right')
ax.set_ylabel('events', loc='top')
ax.set_title('Missing TOF HG hits?', loc='right')
plt.show()

In [ ]:
who

In [ ]:
%matplotlib inline

import charmingbeauty.layout as lo
import numpy as np

cb.visual.set_style_default()
fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
ax  = fig.gca()
ax.set_ylabel('Hz', loc='top')
ax.set_xlabel('MET [s] (gcu)', loc='right')

times   = np.array([j[0] for j in mtbmoni])
times  -= times[0]
rates   = np.array([j[1].tiu_bad for j in mtbmoni])
l_rates = np.array([j[1].tiu_busy_stuck for j in mtbmoni])
ax.plot(times, rates, lw=0.8, alpha=0.7, label='tiu_bad')
ax.plot(times, l_rates, lw=0.8, alpha=0.7, label='tiu_busy_stuck')
ax.legend(loc='upper right', frameon=False)
ax.set_title(f'MTB rates', loc='right')
plt.show()

In [ ]:
%matplotlib inline

import charmingbeauty.layout as lo
import numpy as np

cb.visual.set_style_default()

from collections import defaultdict
rbrates = defaultdict(list)
for k in rbmoni:
    rbrates[k[1].board_id].append((k[0],k[1].rate))

for k in sorted(rbrates):
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
    ax  = fig.gca()
    ax.set_ylabel('Hz', loc='top')
    ax.set_xlabel('MET [s] (gcu)', loc='right')

    times = np.array([j[0] for j in rbrates[k]])
    times -= times[0]
    rates  = np.array([j[1] for j in rbrates[k]])
    ax.plot(times, rates, lw=0.8, alpha=0.7)
    dbrb = go.db.ReadoutBoard.objects.filter(rb_id=k)[0]
    panels = []
    for j in dbrb.paddles:
        panels.append(j.panel_id)
    panels = set(panels)
    ax.set_title(f'RB {k} (panels {panels})', loc='right')

plt.show()

In [ ]:
def good_hits_trk(tracker_event):
    n_good = 0
    rows   = defaultdict(lambda : 0)
    for k in tracker_event.hits:
        if (k.asic_event_code == 0) or (k.asic_event_code == 2):
            n_good += 1
            rows[k.row] += 1
    return n_good, rows


print (layers)

In [ ]:
%matplotlib inline
data = [layers[k] for k in layers]
keys = [k-128 for k in layers]
fig  = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
ax   = fig.gca()
ax.set_title('Good Tracker hits by layer', loc='right')
ax.set_ylabel('layer', loc='top')
ax.set_xlabel('good hits (aev 0,2)', loc='right')
ax.set_xscale('log')
ax.barh(keys,data)
ax.invert_yaxis()
plt.show()

In [ ]:
%matplotlib inline
for k in layer_rows:
    #for r in layer_rows[k]:
    data = [laye    def draw_xz(self) ->:
        """
        Draw a matplotlib patch for xy projection
        """
        pass
r_rows[k][r] for r in layer_rows[k]]
    keys = [j for j in layer_rows[k]]
    fig  = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax   = fig.gca()
    ax.set_title(f'Good Tracker hits for rows layer {k}', loc='right')
    ax.set_xlabel('row', loc='right')
    ax.set_ylabel('good hits (aev 0,2)', loc='top')
    ax.set_yscale('log')
    ax.bar(keys,data)
    #ax.invert_yaxis()
plt.show()

In [ ]:
from collections import defaultdict

def good_hits_trk(tracker_event):
    n_good = 0
    rows   = defaultdict(lambda : 0)
    for k in tracker_event.hits:
        if (k.asic_event_code == 0) or (k.asic_event_code == 2):
            n_good += 1
            rows[k.row] += 1
    return n_good, rows

layer_health      = defaultdict(lambda : [])
layer_rows_health = {k : defaultdict(lambda : 0) for k in range(10)}
for idx,ev in enumerate(tqdm.tqdm(events)):
    if idx % 1000 != 0:
        continue
    for trk_ev in ev.tracker:
        n_good, rows = good_hits_trk(trk_ev)
        if n_good > 0:
            layer_health[trk_ev.layer - 128].append((trk_ev.event_time, n_good))
            #layer_rows_health[trk_ev.layer - 128][tre


In [ ]:

for l in sorted(layer_health):
    fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax  = fig.gca()
    ax.set_xlabel('MET (gcu)', loc='left')
    ax.set_ylabel('nhits', loc='top')
    ax.set_title(f'Layer {l}', loc='right')
    times = [k[0] for k in layer_health[l]]
    #print (times[:10])
    data  = [k[1] for k in layer_health[l]]
    #print (data[:10])
    try:
        ax.scatter(times, data, alpha=0.8)
    except:
        pass
    cb.visual.adjust_minor_ticks(ax, which='both')
plt.show()

In [ ]:
good_hits_trk(ev.tracker[0])
print(len(ev.tracker[0].hits))

ev.tracker[0]

In [ ]:
import dashi as d
d.visual()

fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
ax  = fig.gca()
for k in layers:
    h = d.factory.hist1d(layers[k], np.arange(0,10,1))
    h.line(color='b')
plt.show()

In [ ]:
%matplotlib inline
import charmingbeauty.layout as lo
import numpy as np

rates    = [k[1].rate for k in mtbmoni]
rates_t  = np.array([k[0] for k in mtbmoni])
rates_t  -= rates_t[0]

l_rates   = [k[1].lost_rate for k in mtbmoni]
l_rates_t = np.array([k[0] for k in mtbmoni])
l_rates_t -= l_rates_t[0]

fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)

ax  = fig.gca()
ax.set_title('MTB rates', loc='right')
ax.plot(rates_t,rates, lw=1.2, label='rate', alpha=0.9)
ax.plot(l_rates_t, l_rates, lw=1.2, label='lost rate', alpha=0.6, color='r')
ax.set_ylabel('Hz', loc='top')
ax.set_xlabel('MET [s] (gcu)', loc='right')
ax.legend(frameon=False, loc='upper right')
cb.visual.adjust_minor_ticks(ax, which='both')
plt.show()


print (len(rates))


In [ ]:
cube_top = go.db.Panel.objects.filter(panel_id=1)
cube_top_rbs = []
for panel in cube_top:
    for pdl in panel.paddles:
        cube_top_rbs.append(pdl.rb_id)

cube_rest = go.db.Panel.objects.filter(panel_id__lt=7, panel_id__gt=1)
cube_rest_rbs = []
for panel in cube_rest:
    for pdl in panel.paddles:
        cube_rest_rbs.append(pdl.rb_id)

umbrella = go.db.Panel.objects.filter(panel_id__gt=6, panel_id__lt=14)
umbrella_rbs = []
for panel in umbrella:
    for pdl in panel.paddles:
        umbrella_rbs.append(pdl.rb_id)
        
cortina = go.db.Panel.objects.filter(panel_id__gt=13)
cortina_rbs = []
for panel in cortina:
    for pdl in panel.paddles:
        cortina_rbs.append(pdl.rb_id)

cube_top_rbs = list(set(cube_top_rbs))
cube_rest_rbs = list(set(cube_rbs))
umbrella_rbs = list(set(umbrella_rbs))
cortina_rbs = list(set(cortina_rbs))

print (len(cube_top_rbs))
print (len(cube_rest_rbs))
print (len(umbrella_rbs))
print (len(cortina_rbs))

In [ ]:
%matplotlib inline


fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
ax  = fig.gca()
ax.set_ylabel('Hz', loc='top')
ax.set_xlabel('MET [s] (gcu)', loc='right')
for k in cube_top_rbs:
    if not k in rbrates:
        continue
    times = np.array([j[0] for j in rbrates[k]])
    times -= times[0]
    rates  = np.array([j[1] for j in rbrates[k]])
    ax.plot(times, rates, lw=0.8, alpha=0.7)
ax.set_title('cube top', loc='right')

fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
ax  = fig.gca()
ax.set_ylabel('Hz', loc='top')
ax.set_xlabel('MET [s] (gcu)', loc='right')
for k in cube_rest_rbs:
    if not k in rbrates:
        continue
    times = np.array([j[0] for j in rbrates[k]])
    times -= times[0]
    rates  = np.array([j[1] for j in rbrates[k]])
    ax.plot(times, rates, lw=0.8, alpha=0.7)
ax.set_title('cube, no top', loc='right')

fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
ax  = fig.gca()
ax.set_ylabel('Hz', loc='top')
ax.set_xlabel('MET [s] (gcu)', loc='right')
for k in umbrella_rbs:
    if not k in rbrates:
        continue
    times = np.array([j[0] for j in rbrates[k]])
    times -= times[0]
    rates  = np.array([j[1] for j in rbrates[k]])
    ax.plot(times, rates, lw=0.8, alpha=0.7)
ax.set_title('umbrella', loc='right')

fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
ax  = fig.gca()
ax.set_ylabel('Hz', loc='top')
ax.set_xlabel('MET [s] (gcu)', loc='right')
for k in cortina_rbs:
    if not k in rbrates:
        continue
    times = np.array([j[0] for j in rbrates[k]])
    times -= times[0]
    rates  = np.array([j[1] for j in rbrates[k]])
    ax.plot(times, rates, lw=0.8, alpha=0.7)
ax.set_title('cortina', loc='right')

plt.show()

In [ ]:
40000/3600

In [ ]:
print (mtbmoni[0])

In [ ]:
%matplotlib inline
cb.visual.set_style_present()
plt.plot(evids)
plt.show()

In [ ]:
import matplotlib
import matplotlib.font_manager

In [ ]:
matplotlib.get_cachedir()

In [ ]:
sorted(matplotlib.font_manager.get_font_names())

In [ ]:
ev

In [ ]:
paddles = go.db.get_umbrella_paddles()
dir (paddles[0])

In [ ]:
%matplotlib inline

import charmingbeauty as cb
import matplotlib.patches as patches

cb.visual.set_style_present()
lo = cb.layout
fig = plt.figure(figsize=lo.FIGSIZE_A4_SQUARE)
ax = fig.gca()
pdl_x = [pdl.global_pos_x_l0 for pdl in paddles]
pdl_y = [pdl.global_pos_y_l0 for pdl in paddles]
pdl_x_A = [pdl.global_pos_x_l0_A for pdl in paddles]
pdl_y_A = [pdl.global_pos_y_l0_A for pdl in paddles]
pdl_x_B = [pdl.global_pos_x_l0_B for pdl in paddles]
pdl_y_B = [pdl.global_pos_y_l0_B for pdl in paddles]


pids = [pdl.paddle_id for pdl in paddles]
patches = []
for txt, x, y in zip(pids, pdl_x, pdl_y):
    #f txt == 108:
    #ax.text(x,y,str(txt))#, fontsize=16)
    #break
    pass
for xA,yA,xB,yB in zip(pdl_x_A, pdl_y_A, pdl_x_B, pdl_y_B):
    # the first tuple in the rectangle is the bottom left corner
    corner = []
    if xA < yA: # x is left
        corner=[xA - 
    patches.append(patches.Rectangle(
ax.scatter(pdl_x,pdl_y, s=30, marker='+')
ax.scatter(pdl_x_A, pdl_y_A, s=20, marker='x')
ax.scatter(pdl_x_B, pdl_y_B, s=20, marker='x')
plt.show()

In [ ]:
# tof files
go.